# Lab Cleanup: Remove AgentCore and other lab resources

This notebook helps you clean up all AWS resources created during the Advanced Agentic AI lab. Running these cleanup steps ensures you don't incur unnecessary charges after completing the lab exercises.

**Important**: Run these cells in order to properly clean up all resources.

## Cleanup Overview

During this lab, you created several AWS resources:

1. **AgentCore Runtime** - Serverless agent hosting environment
2. **AgentCore Memory** - Conversation memory storage
3. **Bedrock Guardrail** - Content filtering for responsible AI
4. **Amazon Cognito User Pool** - Authentication for AgentCore Runtime
5. **Amazon ECR Repository** - Container image storage
6. **AWS CodeBuild Project** - Container build automation
7. **IAM Roles** - Execution roles for AgentCore services
8. **S3 Bucket** - CodeBuild source artifacts
9. **CloudWatch Log Groups** - Runtime and build logs

This notebook will systematically delete each resource type to ensure complete cleanup.

### Step 1: Initialize AWS Clients

First, we'll initialize the AWS SDK (boto3) clients needed to interact with each service. This establishes connections to all AWS services we'll use for cleanup.

In [1]:
import boto3
import time

# Initialize AWS clients for each service we need to clean up
bedrock = boto3.client('bedrock', region_name='us-west-2')
cognito = boto3.client('cognito-idp', region_name='us-west-2')
ecr = boto3.client('ecr', region_name='us-west-2')
codebuild = boto3.client('codebuild', region_name='us-west-2')
iam = boto3.client('iam', region_name='us-west-2')
s3 = boto3.client('s3', region_name='us-west-2')
logs = boto3.client('logs', region_name='us-west-2')
agentcore_client = boto3.client('bedrock-agentcore-control', region_name='us-west-2')

print("✅ AWS clients initialized successfully")
print("Ready to begin cleanup process...")

✅ AWS clients initialized successfully
Ready to begin cleanup process...


### Step 2: Delete AgentCore Runtime

The AgentCore Runtime is the serverless environment hosting your deployed agent. This is the primary resource that incurs ongoing charges if left running.

**What this does:**
- Lists all AgentCore Runtimes in your account
- Identifies runtimes matching 'personal_finance_agent'
- Deletes the runtime and stops all agent invocations

**Note**: This uses the `bedrock-agentcore-control` service, which is different from the `bedrock-agent` service used for Bedrock Agents.

In [2]:
# Delete AgentCore Runtime
print("=" * 70)
print("STEP 2: Deleting AgentCore Runtime")
print("=" * 70)

try:
    # List all AgentCore Runtimes
    response = agentcore_client.list_agent_runtimes(maxResults=50)
    
    runtimes_found = response.get('agentRuntimes', [])
    print(f"Found {len(runtimes_found)} AgentCore Runtime(s)")
    
    if not runtimes_found:
        print("⚠️  No AgentCore Runtimes found to delete")
    else:
        # Delete runtimes matching our agent name
        deleted_count = 0
        for runtime in runtimes_found:
            runtime_name = runtime.get('agentRuntimeName', '')
            runtime_id = runtime['agentRuntimeId']
            
            # Check if this is our personal finance agent
            if 'personal_finance_agent' in runtime_name.lower():
                print(f"\nDeleting runtime: {runtime_name}")
                print(f"  Runtime ID: {runtime_id}")
                
                try:
                    agentcore_client.delete_agent_runtime(
                        agentRuntimeId=runtime_id
                    )
                    print(f"  ✅ Successfully deleted: {runtime_name}")
                    deleted_count += 1
                except Exception as delete_error:
                    print(f"  ❌ Error deleting {runtime_name}: {delete_error}")
        
        if deleted_count == 0:
            print("\n⚠️  No runtimes matched 'personal_finance_agent'")
        else:
            print(f"\n✅ Deleted {deleted_count} AgentCore Runtime(s)")

except Exception as e:
    print(f"❌ Error during AgentCore Runtime cleanup: {e}")

print("=" * 70)

STEP 2: Deleting AgentCore Runtime
Found 1 AgentCore Runtime(s)

Deleting runtime: personal_finance_agent
  Runtime ID: personal_finance_agent-uYgRti2HCd
  ✅ Successfully deleted: personal_finance_agent

✅ Deleted 1 AgentCore Runtime(s)


### Step 3: Delete AgentCore Memory

AgentCore Memory stores conversation history and context across sessions. This resource should be deleted to avoid storage charges.

**What this does:**
- Lists all AgentCore Memory resources in your account
- Identifies memories with names starting with 'FinancialAdvisorMemory'
- Deletes the memory resource and all stored conversation data

**Note**: If you didn't complete Task 3.2.1, this step will be skipped automatically.

In [3]:
# Delete AgentCore Memory
print("=" * 70)
print("STEP 3: Deleting AgentCore Memory")
print("=" * 70)

try:
    from bedrock_agentcore.memory import MemoryClient
    memory_client = MemoryClient()
    
    # List all memories and find FinancialAdvisorMemory
    memories = memory_client.list_memories()
    
    if not memories:
        print("⚠️  No AgentCore Memory resources found")
        print("   (This is normal if you didn't complete Task 3.2.1)")
    else:
        print(f"Found {len(memories)} memory resource(s)")
        deleted_count = 0
        
        for memory in memories:
            memory_name = memory.get('name', '')
            memory_id = memory.get('id', '')
            
            # Check if this is our FinancialAdvisorMemory (check both name and id)
            if 'FinancialAdvisorMemory' in memory_name or 'FinancialAdvisorMemory' in memory_id:
                print(f"\nDeleting memory: {memory_name}")
                print(f"  Memory ID: {memory_id}")
                
                try:
                    memory_client.delete_memory(memory_id=memory_id)
                    print(f"  ✅ Successfully deleted: {memory_name}")
                    deleted_count += 1
                except Exception as delete_error:
                    print(f"  ❌ Error deleting {memory_name}: {delete_error}")
        
        if deleted_count == 0:
            print("\n⚠️  No memories matched 'FinancialAdvisorMemory'")
            print("   (This is normal if you didn't complete Task 3.2.1)")
            print("\n   Found memories (for debugging):")
            for memory in memories:
                print(f"     - Name: {memory.get('name', 'N/A')}, ID: {memory.get('id', 'N/A')}")
        else:
            print(f"\n✅ Deleted {deleted_count} AgentCore Memory resource(s)")
        
except ImportError:
    print("⚠️  bedrock_agentcore.memory module not available")
    print("   Skipping memory deletion")
except Exception as e:
    print(f"❌ Error during memory cleanup: {e}")

print("=" * 70)

STEP 3: Deleting AgentCore Memory
Found 1 memory resource(s)

Deleting memory: 
  Memory ID: FinancialAdvisorMemory-segF4298qr
  ✅ Successfully deleted: 

✅ Deleted 1 AgentCore Memory resource(s)


### Step 4: Delete Bedrock Guardrail

The Bedrock Guardrail provides content filtering for responsible AI. It was created in Task 1 to prevent the agent from giving cryptocurrency advice.

**What this does:**
- Lists all Bedrock Guardrails in your account
- Identifies the 'guardrail-no-bitcoin-advice' guardrail
- Deletes the guardrail configuration

In [4]:
# Delete Bedrock Guardrail
print("=" * 70)
print("STEP 4: Deleting Bedrock Guardrail")
print("=" * 70)

try:
    response = bedrock.list_guardrails()
    guardrails_found = response.get('guardrails', [])
    
    if not guardrails_found:
        print("⚠️  No guardrails found")
    else:
        deleted_count = 0
        for guardrail in guardrails_found:
            if 'guardrail-no-bitcoin-advice' in guardrail['name']:
                print(f"Deleting guardrail: {guardrail['name']}")
                print(f"  Guardrail ID: {guardrail['id']}")
                
                bedrock.delete_guardrail(guardrailIdentifier=guardrail['id'])
                print(f"  ✅ Successfully deleted: {guardrail['name']}")
                deleted_count += 1
        
        if deleted_count == 0:
            print("⚠️  No guardrails matched 'guardrail-no-bitcoin-advice'")
        else:
            print(f"✅ Deleted {deleted_count} guardrail(s)")
            
except Exception as e:
    print(f"❌ Error deleting guardrail: {e}")

print("=" * 70)

STEP 4: Deleting Bedrock Guardrail
Deleting guardrail: guardrail-no-bitcoin-advice
  Guardrail ID: sq4faz0bh9ku
  ✅ Successfully deleted: guardrail-no-bitcoin-advice
✅ Deleted 1 guardrail(s)


### Step 5: Delete Amazon Cognito User Pool

The Cognito User Pool provides authentication for AgentCore Runtime invocations. It was created in Task 3.2 to generate JWT tokens.

**What this does:**
- Lists all Cognito User Pools in your account
- Identifies pools named 'agentpool'
- Deletes the user pool and all associated users

**Note**: This also removes the credentials stored in AWS Secrets Manager.

In [5]:
# Delete Cognito User Pool
print("=" * 70)
print("STEP 5: Deleting Amazon Cognito User Pool")
print("=" * 70)

try:
    response = cognito.list_user_pools(MaxResults=60)
    pools_found = response.get('UserPools', [])
    
    if not pools_found:
        print("⚠️  No user pools found")
    else:
        deleted_count = 0
        for pool in pools_found:
            if 'agentpool' in pool['Name'].lower():
                print(f"Deleting user pool: {pool['Name']}")
                print(f"  Pool ID: {pool['Id']}")
                
                cognito.delete_user_pool(UserPoolId=pool['Id'])
                print(f"  ✅ Successfully deleted: {pool['Name']}")
                deleted_count += 1
        
        if deleted_count == 0:
            print("⚠️  No user pools matched 'agentpool'")
        else:
            print(f"✅ Deleted {deleted_count} user pool(s)")
            
except Exception as e:
    print(f"❌ Error deleting Cognito user pool: {e}")

print("=" * 70)

STEP 5: Deleting Amazon Cognito User Pool
Deleting user pool: agentpool
  Pool ID: us-west-2_G7mI5C1LN
  ✅ Successfully deleted: agentpool
✅ Deleted 1 user pool(s)


### Step 6: Delete Amazon ECR Repository

The ECR (Elastic Container Registry) repository stores the Docker container image for your agent. This was created automatically by the AgentCore SDK during deployment.

**What this does:**
- Lists all ECR repositories in your account
- Identifies repositories containing 'bedrock-agentcore-personal_finance_agent'
- Deletes all container images in the repository
- Deletes the repository itself

**Note**: The `force=True` parameter ensures all images are deleted even if they're in use.

In [6]:
# Delete ECR Repository
print("=" * 70)
print("STEP 6: Deleting Amazon ECR Repository")
print("=" * 70)

try:
    response = ecr.describe_repositories()
    repos_found = response.get('repositories', [])
    
    if not repos_found:
        print("⚠️  No ECR repositories found")
    else:
        deleted_count = 0
        for repo in repos_found:
            repo_name = repo['repositoryName']
            
            if 'bedrock-agentcore-personal_finance_agent' in repo_name:
                print(f"Deleting ECR repository: {repo_name}")
                print(f"  Repository URI: {repo['repositoryUri']}")
                
                # Delete repository with force=True to remove all images
                ecr.delete_repository(
                    repositoryName=repo_name,
                    force=True
                )
                print(f"  ✅ Successfully deleted: {repo_name}")
                deleted_count += 1
        
        if deleted_count == 0:
            print("⚠️  No repositories matched 'bedrock-agentcore-personal_finance_agent'")
        else:
            print(f"✅ Deleted {deleted_count} ECR repository(ies)")
            
except Exception as e:
    print(f"❌ Error deleting ECR repository: {e}")

print("=" * 70)

STEP 6: Deleting Amazon ECR Repository
Deleting ECR repository: bedrock-agentcore-personal_finance_agent
  Repository URI: 011673140073.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-personal_finance_agent
  ✅ Successfully deleted: bedrock-agentcore-personal_finance_agent
✅ Deleted 1 ECR repository(ies)


### Step 7: Delete AWS CodeBuild Project

The CodeBuild project was created by the AgentCore SDK to build and push your agent's Docker container image to ECR.

**What this does:**
- Lists all CodeBuild projects in your account
- Identifies projects containing 'bedrock-agentcore-personal_finance_agent-builder'
- Deletes the CodeBuild project configuration

**Note**: If you encounter permission errors, you can safely skip this step and delete the project manually through the AWS Console.

In [7]:
# Delete CodeBuild Project
print("=" * 70)
print("STEP 7: Deleting AWS CodeBuild Project")
print("=" * 70)

try:
    response = codebuild.list_projects()
    projects_found = response.get('projects', [])
    
    if not projects_found:
        print("⚠️  No CodeBuild projects found")
    else:
        deleted_count = 0
        for project in projects_found:
            if 'bedrock-agentcore-personal_finance_agent-builder' in project:
                print(f"Deleting CodeBuild project: {project}")
                
                codebuild.delete_project(name=project)
                print(f"  ✅ Successfully deleted: {project}")
                deleted_count += 1
        
        if deleted_count == 0:
            print("⚠️  No projects matched 'bedrock-agentcore-personal_finance_agent-builder'")
        else:
            print(f"✅ Deleted {deleted_count} CodeBuild project(s)")
            
except Exception as e:
    if 'AccessDenied' in str(e):
        print("⚠️  Insufficient permissions to delete CodeBuild project")
        print("   You can delete it manually via AWS Console if needed")
    else:
        print(f"❌ Error deleting CodeBuild project: {e}")

print("=" * 70)

STEP 7: Deleting AWS CodeBuild Project
Deleting CodeBuild project: bedrock-agentcore-personal_finance_agent-builder
  ✅ Successfully deleted: bedrock-agentcore-personal_finance_agent-builder
✅ Deleted 1 CodeBuild project(s)


### Step 8: Delete IAM Roles

The AgentCore SDK automatically creates IAM roles for runtime execution and CodeBuild operations. These roles need to be cleaned up to avoid clutter in your IAM console.

**What this does:**
- Lists all IAM roles in your account
- Identifies roles starting with 'AmazonBedrockAgentCoreSDKRuntime-' or 'AmazonBedrockAgentCoreSDKCodeBuild-'
- Detaches all managed policies from each role
- Deletes all inline policies from each role
- Deletes the IAM role itself

**Note**: Roles must have all policies removed before they can be deleted.

In [8]:
# Delete IAM Roles
print("=" * 70)
print("STEP 8: Deleting IAM Roles")
print("=" * 70)

def delete_iam_role(role_name):
    """Helper function to completely delete an IAM role with all its policies"""
    try:
        print(f"Deleting IAM role: {role_name}")
        
        # Step 1: Detach all managed policies
        attached = iam.list_attached_role_policies(RoleName=role_name)
        for policy in attached['AttachedPolicies']:
            iam.detach_role_policy(
                RoleName=role_name,
                PolicyArn=policy['PolicyArn']
            )
            print(f"  Detached managed policy: {policy['PolicyName']}")
        
        # Step 2: Delete all inline policies
        inline = iam.list_role_policies(RoleName=role_name)
        for policy_name in inline['PolicyNames']:
            iam.delete_role_policy(
                RoleName=role_name,
                PolicyName=policy_name
            )
            print(f"  Deleted inline policy: {policy_name}")
        
        # Step 3: Delete the role
        iam.delete_role(RoleName=role_name)
        print(f"  ✅ Successfully deleted role: {role_name}")
        return True
        
    except Exception as e:
        print(f"  ❌ Error deleting role {role_name}: {e}")
        return False

try:
    response = iam.list_roles()
    roles_found = response.get('Roles', [])
    
    deleted_count = 0
    for role in roles_found:
        role_name = role['RoleName']
        
        # Check if this is an AgentCore SDK role
        if role_name.startswith('AmazonBedrockAgentCoreSDKRuntime-') or \
           role_name.startswith('AmazonBedrockAgentCoreSDKCodeBuild-'):
            if delete_iam_role(role_name):
                deleted_count += 1
    
    if deleted_count == 0:
        print("⚠️  No AgentCore SDK IAM roles found")
    else:
        print(f"\n✅ Deleted {deleted_count} IAM role(s)")
        
except Exception as e:
    if 'AccessDenied' in str(e):
        print("⚠️  Insufficient permissions to delete IAM roles")
        print("   You can delete them manually via AWS Console if needed")
    else:
        print(f"❌ Error listing IAM roles: {e}")

print("=" * 70)

STEP 8: Deleting IAM Roles
Deleting IAM role: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-5d12c2867b
  Deleted inline policy: CodeBuildExecutionPolicy
  ✅ Successfully deleted role: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-5d12c2867b
Deleting IAM role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-5d12c2867b
  Deleted inline policy: BedrockAgentCoreRuntimeExecutionPolicy-personal_finance_agent
  ✅ Successfully deleted role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-5d12c2867b

✅ Deleted 2 IAM role(s)


### Step 9: Delete S3 Bucket

The S3 bucket stores source code artifacts used by CodeBuild to build your agent's container image. This bucket must be completely emptied before it can be deleted.

**What this does:**
- Lists all S3 buckets in your account
- Identifies buckets starting with 'bedrock-agentcore-codebuild-sources-'
- Deletes all object versions (if versioning is enabled)
- Deletes all delete markers
- Deletes all current objects
- Deletes the empty bucket

**Note**: This process can take a few moments if the bucket contains many objects.

In [9]:
# Delete S3 Bucket
print("=" * 70)
print("STEP 9: Deleting S3 Bucket")
print("=" * 70)

try:
    response = s3.list_buckets()
    buckets_found = response.get('Buckets', [])
    
    deleted_count = 0
    for bucket in buckets_found:
        bucket_name = bucket['Name']
        
        if bucket_name.startswith('bedrock-agentcore-codebuild-sources-'):
            print(f"Deleting S3 bucket: {bucket_name}")
            
            # Step 1: Empty the bucket completely
            try:
                print("  Emptying bucket...")
                
                # Delete all object versions (if versioning enabled)
                paginator = s3.get_paginator('list_object_versions')
                for page in paginator.paginate(Bucket=bucket_name):
                    objects = []
                    
                    # Collect all versions
                    for version in page.get('Versions', []):
                        objects.append({
                            'Key': version['Key'],
                            'VersionId': version['VersionId']
                        })
                    
                    # Collect all delete markers
                    for marker in page.get('DeleteMarkers', []):
                        objects.append({
                            'Key': marker['Key'],
                            'VersionId': marker['VersionId']
                        })
                    
                    # Delete in batches
                    if objects:
                        s3.delete_objects(
                            Bucket=bucket_name,
                            Delete={'Objects': objects}
                        )
                        print(f"    Deleted {len(objects)} object version(s)")
                
                # Delete current objects (non-versioned)
                paginator = s3.get_paginator('list_objects_v2')
                for page in paginator.paginate(Bucket=bucket_name):
                    objects = [{'Key': obj['Key']} for obj in page.get('Contents', [])]
                    if objects:
                        s3.delete_objects(
                            Bucket=bucket_name,
                            Delete={'Objects': objects}
                        )
                        print(f"    Deleted {len(objects)} object(s)")
                
            except Exception as empty_error:
                print(f"  ⚠️  Error emptying bucket: {empty_error}")
                print(f"  Skipping bucket deletion")
                continue
            
            # Step 2: Delete the empty bucket
            s3.delete_bucket(Bucket=bucket_name)
            print(f"  ✅ Successfully deleted bucket: {bucket_name}")
            deleted_count += 1
    
    if deleted_count == 0:
        print("⚠️  No buckets matched 'bedrock-agentcore-codebuild-sources-'")
    else:
        print(f"\n✅ Deleted {deleted_count} S3 bucket(s)")
        
except Exception as e:
    print(f"❌ Error deleting S3 bucket: {e}")

print("=" * 70)

STEP 9: Deleting S3 Bucket
Deleting S3 bucket: bedrock-agentcore-codebuild-sources-011673140073-us-west-2
  Emptying bucket...
    Deleted 1 object version(s)
  ✅ Successfully deleted bucket: bedrock-agentcore-codebuild-sources-011673140073-us-west-2

✅ Deleted 1 S3 bucket(s)


### Step 10: Delete CloudWatch Log Groups

CloudWatch Log Groups store logs from your AgentCore Runtime, CodeBuild builds, and Lambda functions. These should be deleted to avoid storage charges.

**What this does:**
- Searches for log groups with specific prefixes:
  - `/aws/bedrock-agentcore/runtimes/` - AgentCore Runtime logs
  - `/aws/codebuild/bedrock-agentcore` - CodeBuild build logs
  - `/aws/lambda/AdvancedAgenticAI-Lab` - Lambda function logs
- Deletes all matching log groups and their log streams

**Note**: This is the final cleanup step. After this, all lab resources should be removed.

In [10]:
# Delete CloudWatch Log Groups
print("=" * 70)
print("STEP 10: Deleting CloudWatch Log Groups")
print("=" * 70)

# Define log group prefixes to search for
patterns = [
    '/aws/bedrock-agentcore/runtimes/personal_finance_agent',
    '/aws/codebuild/bedrock-agentcore',
    '/aws/lambda/AdvancedAgenticAI-Lab'
]

total_deleted = 0

for pattern in patterns:
    try:
        print(f"\nSearching for log groups with prefix: {pattern}")
        response = logs.describe_log_groups(logGroupNamePrefix=pattern)
        log_groups_found = response.get('logGroups', [])
        
        if not log_groups_found:
            print(f"  ⚠️  No log groups found with prefix: {pattern}")
        else:
            for log_group in log_groups_found:
                log_group_name = log_group['logGroupName']
                print(f"  Deleting log group: {log_group_name}")
                
                logs.delete_log_group(logGroupName=log_group_name)
                print(f"    ✅ Successfully deleted")
                total_deleted += 1
                
    except Exception as e:
        if 'AccessDenied' in str(e):
            print(f"  ⚠️  Insufficient permissions for pattern: {pattern}")
            print(f"     You can delete these manually via AWS Console if needed")
        else:
            print(f"  ❌ Error deleting log groups with pattern {pattern}: {e}")

if total_deleted > 0:
    print(f"\n✅ Deleted {total_deleted} log group(s) total")
else:
    print(f"\n⚠️  No log groups were deleted")

print("=" * 70)

STEP 10: Deleting CloudWatch Log Groups

Searching for log groups with prefix: /aws/bedrock-agentcore/runtimes/personal_finance_agent
  Deleting log group: /aws/bedrock-agentcore/runtimes/personal_finance_agent-uYgRti2HCd-DEFAULT
    ✅ Successfully deleted

Searching for log groups with prefix: /aws/codebuild/bedrock-agentcore
  Deleting log group: /aws/codebuild/bedrock-agentcore-personal_finance_agent-builder
    ✅ Successfully deleted
  Deleting log group: /aws/codebuild/bedrock-agentcore-weather_agent-builder
    ✅ Successfully deleted

Searching for log groups with prefix: /aws/lambda/AdvancedAgenticAI-Lab
  ⚠️  No log groups found with prefix: /aws/lambda/AdvancedAgenticAI-Lab

✅ Deleted 3 log group(s) total


## Cleanup Complete!

You have successfully cleaned up all AWS resources created during the Advanced Agentic AI lab.

### Resources Deleted:

✅ AgentCore Runtime - Serverless agent hosting  
✅ AgentCore Memory - Conversation storage  
✅ Bedrock Guardrail - Content filtering  
✅ Cognito User Pool - Authentication  
✅ ECR Repository - Container images  
✅ CodeBuild Project - Build automation  
✅ IAM Roles - Execution roles  
✅ S3 Bucket - Build artifacts  
✅ CloudWatch Log Groups - Runtime logs  

### Next Steps:

1. **Verify Cleanup**: Check the AWS Console to confirm all resources are deleted
2. **Review Costs**: Monitor your AWS billing dashboard for any remaining charges
3. **Manual Cleanup**: If any resources showed permission errors, delete them manually via the AWS Console

To continue, return to the lab instructions and proceed to the **Delete CloudFormation stack** section.